In [6]:
import numpy as np
import os
import pandas as pd
from tqdm import tqdm

notebook_dir = os.path.dirname(os.path.abspath('__file__'))
os.chdir(notebook_dir)

df = pd.read_csv('ADNI_merged_data_clean.csv')

In [2]:
covariates = df.columns.tolist()
#print(covariates)
vital_var = ['VITALweight', 'VITALheight', 'VITALbloodpressDIA', 'VITALbloodpressSYS', 'VITALheartrate']
blood_var = ['ALT', 'Alkaline_Pho', 'AST','Cholesterol','Creatine_kinase', 'Creatinine', 'Eosinophils', 'GGT', 'Glucose', 'Hematocrit', 'Hemoglobin', 'Lymphocytes', 'Monocytes', 'Platelets', 'Vitamin_B12', 'Triglycerides', 'Indirect_Bilirubin']
demo_var = ['PTRACCAT', 'PTGENDER', 'PTEDUCAT', 'Age']
ADAS_var = ['ADASword_recall', 'ADAScommands', 'ADASconstruction', 'ADASdelayed_word_recall', 'ADASnaming', 'ADASideational', 'ADASorientation', 'ADASword_recognation', 'ADASrem_instruction', 'ADAScomprehension', 'ADASword_finding', 'ADASspoken_language', 'ADAScancellation']
MMSE_var = ['MMSE_orient', 'MMSE_regist', 'MMSE_attent', 'MMSE_recall', 'MMSE_language']
CD_var =  ['CDMEMORY', 'CDORIENT', 'CDJUDGE', 'CDCOMMUN', 'CDHOME', 'CDCARE']
misc_var = ['HMHYPERT', 'APOE4COUNT', 'TAU', 'PTAU']

static_var = [blood_var, demo_var, misc_var]
t_var = [ADAS_var, MMSE_var, CD_var]


In [3]:
# find the first 1000 patients 
df_first1000 = df[df['RID'].isin(df['RID'].unique()[:1000])]

In [4]:
# using blood var as static var, MMSE m0 as t0 var, MMSE m6 as t1 var in our training set

train_set = []

for rid in df_first1000['RID'].unique():
    baseline = df_first1000[(df_first1000['RID'] == 2) & (df_first1000['VISCODE2'] == 'm0')][blood_var].values.tolist()[0]
    t0 = df_first1000[(df_first1000['RID'] == 2) & (df_first1000['VISCODE2'] == 'm0')][MMSE_var].values.tolist()[0]
    t1 = df_first1000[(df_first1000['RID'] == 2) & (df_first1000['VISCODE2'] == 'm06')][MMSE_var].values.tolist()[0]
    toAppend = baseline+t0+t1
    
    train_set.append(toAppend)



In [7]:
# Sigmoid function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# visible nodes: baseline var, t0 var, t1 var
node_visible = len(train_set[0])

# hidden nodes: randomly chosen number (improve later with cross validation)
node_hidden = 10

# decide how much to update weight/bias each step
learning_rate = 0.1

# Initialize weights/bias
# weight = interaction
#[W_11,W_21,....,W_18_1,......W_23_1] -> hidden 1
#[W_13,W_23,....,W_18_2,......W_23_2] -> hidden 2
#[W_13,W_23,....,W_18_3,......W_23_3] -> hidden 3
#.................................... -> more hidden
# sva1,sva2,....,t0va1,.......t1va1,...

W = np.full((node_hidden,node_visible),0.1)

# bias = score for each individual node
visible_bias = np.zeros(node_visible)
hidden_bias = np.zeros(node_hidden)


# contrastive divergence

epoch = 1000

for i in tqdm(range(epoch)):
    for scenario in train_set:

        # increase existing pattern's probability
            # use gibb's sampling
            # choose a possible scenario matching current scenario:
        hidden_prob = np.sum(W*scenario,axis = 1)

        chosen_hidden = []

        for j in range(node_hidden):
            chosen_hidden.append(np.random.binomial(1, sigmoid(hidden_bias[j]+hidden_prob[j])))
        
        
        # decrease a random pattern's probability
        
        # Negative phase
        neg_scenario = np.random.binomial(n=1 , p = 1/node_visible, size = node_visible)
        
        neg_hidden = np.random.binomial(n=1, p = 1/node_hidden, size = node_hidden)
        

        # update bias/weight according to scenario chosen
        for k in range(len(scenario)):
            if scenario[k] == 1:
                visible_bias[k] += learning_rate
            if neg_scenario[k] == 1:
                visible_bias[k] -= learning_rate
            for l in range(len(chosen_hidden)):
                if scenario[k] == 1 and chosen_hidden[l] == 1:
                    W[l,k] += learning_rate
                if neg_scenario[k] == 1 and neg_hidden[l] == 1:
                    W[l,k] -= learning_rate

        for m in range(len(chosen_hidden)):
            if chosen_hidden[m] == 1:
                hidden_bias[m] += learning_rate
            if neg_hidden[m] == 1:
                hidden_bias[m] -= learning_rate

        

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 1/1000 [00:00<02:31,  6.58it/s]C:\Users\Teddywolf\AppData\Local\Temp\ipykernel_40476\2878621193.py:3: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
100%|██████████| 1000/1000 [02:33<00:00,  6.53it/s]


In [9]:
print(hidden_bias)
print(visible_bias)

[ -9947.20000002  -9964.90000002  -9911.30000002  -9969.30000002
 -10002.70000002 -10010.30000002  -9959.30000002  -9960.00000002
  -9907.10000002 -10006.90000002]
[-3726.1        -3722.8        -3696.7        -3672.7
 -3690.8        -3723.         -3690.6        -3729.8
 -3700.5        -3680.1        -3686.3        -3729.4
 -3706.3        -3703.6        -3713.         -3710.1
 -3682.         -3702.2        -3696.         -3695.3
 96291.10000112 -3714.8        -3728.         -3735.1
 -3691.4        -3728.6        -3699.4       ]


In [ ]:
# to do next:
# generate energy function according to weight and bias.
# Energy = ?

# predict prob(xt+1 | xt, xs) = 1/Z * integral dz e^-energy

# implement GAN to try to distinguish digital twin/ true patient

# ...

ways to improve performance:
cross validation for epoch, hidden node number, weight for gan, learning rate; normalization of variable, 